In [ ]:
import pandas as pd
from scipy import stats
import numpy as np

tidy = pd.read_csv('../data/processed/tidy_data.csv')

def cohens_d(x, y):
    nx, ny = len(x), len(y)
    s_p = np.sqrt(((nx-1)*x.var(ddof=1) + (ny-1)*y.var(ddof=1)) / (nx+ny-2))
    return (x.mean() - y.mean()) / s_p

def print_ttest(var):
    a = tidy[tidy['condition']=='A'][var].dropna()
    b = tidy[tidy['condition']=='B'][var].dropna()

    # Shapiro
    SW_a = stats.shapiro(a)
    SW_b = stats.shapiro(b)
    # Levene
    lev = stats.levene(a, b)

    # t-test (Welch if Levene p<.05)
    if lev.pvalue < 0.05:
        t = stats.ttest_ind(a, b, equal_var=False)
        equal_var = False
    else:
        t = stats.ttest_ind(a, b, equal_var=True)
        equal_var = True

    d = cohens_d(a, b)

    print(f'=== {var} ===')
    print(f'Shapiro A p={SW_a.pvalue:.3f}, B p={SW_b.pvalue:.3f}')
    print(f"Levene p={lev.pvalue:.3f}  (equal_var={equal_var})")
    print(f"t={t.statistic:.2f}, p={t.pvalue:.3f}, Cohen's d={d:.2f}\n")

for v in ['trust_post','mean_rt','accept_rate']:
    print_ttest(v)
